In [1]:
import os
import win32gui
import pandas as pd
import xlwings as xw
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
import pandas as pd

Code summary V1 (first form)

In [15]:
for file in xlsx_files:
    file_path = join(mypath, file)
    wb = load_workbook(file_path)
    ws = wb['Payment Term History']
    ws = wb.active
    try:
        df = pd.read_excel(file_path, sheet_name='Payment Term History',  skiprows=4, usecols=lambda x: 'Unnamed: 0' not in x)
    except ValueError as e:
        continue  
    df = df.dropna(subset=['pam_code'])

    if 'choose' in df['pam_code'].values:
        first_choose_index = df[df['pam_code'] == 'choose'].index[0]
        df = df.loc[:first_choose_index - 1]
    
    main_df = pd.concat([main_df, df], ignore_index=True)
main_df.to_excel(join(mypath, 'append_g4.xlsx'), index=False)

Code summary V2 (specific location)

In [ ]:
# Define the directory containing the Excel files
folder_path = r"C:\Pam_card\g1"
main_data = r"C:\Pam_card\_Data_CutOff_v14.xlsx"

# Open the main Excel file
main_file = xw.Book(main_data, update_links=True)

# Initialize a new Excel workbook to store the combined data
combined_wb = xw.Book()
combined_ws = combined_wb.sheets.active

# Initialize row counter for the destination worksheet
dest_row = 1

# Iterate through each Excel file in the directory
file_list = [filename for filename in os.listdir(folder_path) if filename.startswith("StatementCard_") and filename.endswith(".xlsx")]
total_files = len(file_list)

# Add a progress bar using tqdm
for idx, filename in tqdm(enumerate(file_list, start=1), total=total_files, desc="Copying data"):
    # Load the Excel file
    file_path = os.path.join(folder_path, filename)
    
    try:
        # Open the Excel file
        wb = xw.Book(file_path)
        ws = wb.sheets['Payment Term History']
        
        # Read the value from cell I3
        print(ws.range(3,9).value)
        cell_value = int(ws.range(3,9).value)

        for i in range(7, 7 + cell_value):
            for j in range(1, 134):
                # Get the value from the source worksheet
                cell_collect = ws.cells(i, j).value
                
                # Paste the value to the combined worksheet
                combined_ws.cells(dest_row, j).value = cell_collect
        
            # Update the destination row counter
            dest_row += 1
        
        # Close the workbook
        wb.close()
    
    except (ValueError, IndexError) as e:
        print(f"Error reading file {filename}: {e}")
        continue

# Close the main Excel file after the loop ends
main_file.close()

# Save the combined workbook
output_file = r"C:\Pam_card\combined_data_g1.xlsx"
combined_wb.save(output_file)
combined_wb.close()

print("Data extraction and combination completed.")
   
# Quit Excel application
xw.apps.active.quit()

Code summary V3 (iterate all folder)

In [ ]:
# Define a function to process each Excel file
def process_excel_file(file_path):
    try:
        # Open the Excel file
        wb = xw.Book(file_path)
        ws = wb.sheets['Payment Term History']
        
        # Read the value from cell I3
        cell_value = ws.range(3, 9).value
        if cell_value is None:
            print(f"Cell I3 in file {file_path} is empty.")
            return None

        # Convert cell_value to an integer
        cell_value = int(cell_value)

        # Initialize a list to store cell values
        cell_values = []

        for i in range(7, 7 + cell_value):
            # Construct the range representing the entire row from column A to ED
            row_range = ws.range(f'A{i}:ED{i}')
            row_values = row_range.value
            cell_values.append(row_values)

        # Close the workbook
        wb.close()

        return cell_values

    except (ValueError, IndexError) as e:
        print(f"Error reading file {file_path}: {e}")
        return None

# Define the base directory containing the Excel files
base_folder = r"C:\Pam_card\Audit"
subfolders = ["g1", "g2", "g3", "g4", "g5"]
main_data = r"C:\Pam_card\_Data_CutOff_v14.xlsx"

# Iterate over each subfolder
for subfolder in subfolders:
    # Define the full folder path
    folder_path = os.path.join(base_folder, subfolder)
    
    # Define the output file path for the combined data
    output_file = os.path.join("C:\Pam_card", f"combined_data_{subfolder}.xlsx")

    # Open the main Excel file
    main_file = xw.Book(main_data, update_links=True)

    # Collect file paths for Excel files in the current subfolder
    file_list = [os.path.join(folder_path, filename) for filename in os.listdir(folder_path) if filename.startswith("StatementCard_") and filename.endswith(".xlsx")]
    total_files = len(file_list)
    dest_row = 1
    
    # Initialize a new Excel workbook to store the combined data
    combined_wb = xw.Book()
    combined_ws = combined_wb.sheets.active

    # Process files using multithreading
    with ThreadPoolExecutor(max_workers=2) as executor:
        futures = [executor.submit(process_excel_file, file_path) for file_path in file_list]
        
        # Add a progress bar using tqdm
        for future in tqdm(futures, total=len(file_list), desc=f"Processing {subfolder}"):
            cell_values = future.result()
            if cell_values:
                for row_values in cell_values:
                    combined_ws.range(f'A{dest_row}:ED{dest_row}').value = row_values
                    dest_row += 1  # Increment dest_row only once per row insertion
                    
    xw.apps.active.api.CutCopyMode = False
    
    # Save the combined workbook
    combined_wb.save(output_file)
    combined_wb.close()

print("Data extraction and combination completed.")

# Quit Excel application
xw.apps.active.quit()

In [2]:
# Define a function to copy and paste data to a specific Excel file
def copy_and_paste_data(file_path, data_to_copy):
    try:
        # Open the Excel file
        wb = xw.Book()
        ws = wb.sheets['Sheet1']  # Change to the target sheet name

        # Find the next available row in the target sheet
        next_row = ws.range("A" + str(ws.cells.last_cell.row)).end('up').row + 1

        # Paste the data into the target sheet
        ws.range(f'A{next_row}').value = data_to_copy

        # Close the workbook
        wb.save(file_path)
        wb.close()

    except Exception as e:
        print(f"Error copying and pasting data to file {file_path}: {e}")

# Define a function to process each Excel file
def process_excel_file(file_path):
    try:
        # Open the Excel file
        wb = xw.Book(file_path)
        ws = wb.sheets['Payment Term History']
        
        # Read the value from cell I3
        cell_count = ws.range(3, 9).value
        if cell_count is None:
            print(f"Cell I3 in file {file_path} is empty.")
            return None

        # Convert cell_value to an integer
        cell_count = int(cell_count)

        # Initialize a list to store cell values
        cell_values = []

        for i in range(7, 7 + cell_count):
            # Construct the range representing the entire row from column A to ED
            row_range = ws.range(f'A{i}:ED{i}').value
            cell_values.append(row_range)

        # Close the workbook
        wb.close()

        return cell_values

    except Exception as e:
        print(f"Error reading file {file_path}: {e}")
        return None

def process_excel_file_pandas(file_path):
    try:
        # Read only the necessary cell to get the row count from I3
        cell_value_df = pd.read_excel(file_path, sheet_name='Payment Term History', 
                                      usecols="I", skiprows=2, nrows=1, header=None)
        cell_count = cell_value_df.iloc[0, 0]
        
        if pd.isna(cell_count) or cell_count == "":
            print(f"Cell I3 in file {file_path} is empty.")
            return None
        
        try:
            cell_count = int(cell_count)
        except ValueError:
            print(f"Value in cell I3 of file {file_path} is not a valid integer.")
            return None
        
        # Read the specified number of rows
        rows_df = pd.read_excel(file_path, sheet_name='Payment Term History', 
                                skiprows=6, nrows=cell_count, 
                                usecols="A:ED")
        
        # Option 1: Replace empty strings with a fixed value (e.g., 0, None)
        rows_df.replace("", "Blank", inplace=True)
        
        # Option 2: Drop rows where all elements are empty strings
        # rows_df.dropna(how='all', inplace=True)
        
        # Option 3: Drop rows where any element is an empty string
        # rows_df.dropna(inplace=True)
        
        # Convert DataFrame to a list of rows
        cell_values = rows_df.values.tolist()

        return cell_values

    except Exception as e:
        print(f"Error reading file {file_path}: {e}")
        return None

# Define the base directory containing the Excel files
base_folder = r"C:\Pam_card\update 2024"
subfolders = ["card recheck"]  # You can add more subfolders here "g1", "g2", "g3", "g4", 
main_data = r"C:\Pam_card\_Data_CutOff_v15.xlsx"

# Open the main Excel file
main_file = xw.Book(main_data, update_links=True)

# Get Excel application from window handle
xl_app = xw.apps.active

# Iterate over each subfolder
for subfolder in subfolders:
    # Define the full folder path
    folder_path = os.path.join(base_folder, subfolder)
    
    # Define the output file path for the combined data
    output_file = os.path.join(r"C:\Pam_card\update 2024", f"combined_data_{subfolder}.xlsx")

    # Collect file paths for Excel files in the current subfolder
    file_list = [os.path.join(folder_path, filename) for filename in os.listdir(folder_path) if filename.startswith("StatementCard_") and filename.endswith(".xlsx")]
    total_files = len(file_list)
    dest_row = 1
    
    # Initialize a list to store data from all files
    all_data = []

    # Process files using multithreading
    with ThreadPoolExecutor(max_workers=1) as executor:
        futures = [executor.submit(process_excel_file, file_path) for file_path in file_list]
        
        # Add a progress bar using tqdm
        for future in tqdm(futures, total=len(file_list), desc=f"Processing {subfolder}"):
            data = future.result()
            if data:
                all_data.extend(data)

    # Copy and paste data to the output file
    copy_and_paste_data(output_file, all_data)

print("Data extraction and combination completed.")

# Quit Excel application
xl_app.quit()

Processing card recheck: 100%|███████████████████████████████████████████████████████| 697/697 [08:58<00:00,  1.29it/s]


Data extraction and combination completed.


In [ ]:
#locate interpreter
"C:\Users\Paramut.c\AppData\Local\Programs\Python\Python311\python.exe"

            #row_values = []
            #for j in range(1, 134):
                # Get the value from the source worksheet
                #cell_value = ws.cells(i, j).value
                #row_values.append(cell_value)
            

                #for row, row_values in enumerate(cell_values, start=dest_row):
                    #for col, cell_value in enumerate(row_values, start=1):
                        #combined_ws.cells(row, col).value = cell_value
                #dest_row += len(cell_values)